# Generate Data

Generate restricted IOD data based on the following parameters:

- $a$: Semi-major axis
- $e$: Eccentricity
- $\omega$: Argument of Periapsis
- $M$: Mean Anomaly *(Although in this instance, because of super low eccentricity, it could also be true anomaly)*

In [ ]:
%reload_ext autoreload
%autoreload 2

from data.generator import INITIAL_ORBIT_PARAMS
from data.generator import generate_initial_orbit_dataset_restricted

observer, samples = generate_initial_orbit_dataset_restricted(INITIAL_ORBIT_PARAMS, 1000)
samples = samples.reshape(1000,3)

print("test")



tensor([4.2164e+04, 1.5000e-02, 0.0000e+00, 0.0000e+00])
tensor([[1.0000e+04, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.0000e-06, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 8.1000e+03, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 8.1000e+03]])


In [2]:
import numpy as np
import torch
from astropy.coordinates import ICRS, ITRS, CartesianRepresentation, EarthLocation
from astropy.time import Time
import astropy.units as u
from skyfield.framelib import itrs

current_time = Time.now()

GD_LAT = 0.43157
GD_LONG = 273.085549
alt = 192.5

GD_LAT = np.deg2rad(GD_LAT)
GD_LONG = np.deg2rad(GD_LONG)
alt = alt * 1e-3

distance_to_surface = 6378.137
f = 1.0 / 298.257223563

# Earth Eccentricity
e = np.sqrt((2 * f) - (f ** 2))

# Geodedic Radius
N = distance_to_surface / np.sqrt(1 - (e ** 2) * (np.sin(GD_LAT) ** 2))

# Calculate ECEF Position
R_ECEF = torch.Tensor([
    (N + alt) * np.cos(GD_LAT) * np.cos(GD_LONG),
    (N + alt) * np.cos(GD_LAT) * np.sin(GD_LONG),
    (N * (1 - (e ** 2)) + alt) * np.sin(GD_LAT)
])



arms_itrs = EarthLocation(
    x=R_ECEF[0] * u.km,
    y=R_ECEF[1] * u.km,
    z=R_ECEF[2] * u.km
)

r_ecef = arms_itrs.get_gcrs(obstime=current_time).represent_as('cartesian')
xyz = r_ecef.get_xyz()
xyz = xyz.value
xyz = torch.Tensor(xyz)

print(arms_itrs)
print(r_ecef)
print(xyz)

#ARMS_ITRS = ITRS(R_ECEF_units, representation_type='cartesian', obstime=current_time)
#ARMS_J2K = ARMS_ITRS.transform_to(ICRS())
#ARMS_J2K = ARMS_J2K.represent_as('cartesian')
#print(ARMS_ITRS)
#print(ARMS_J2K)


(343.3164367675781, -6368.9033203125, 47.7215461730957) km
(4990.10303003, -3972.48818789, 35.81022118) km
tensor([ 4990.1030, -3972.4883,    35.8102])
